# SENG 550 Final Project 

Luka Petrovic UCID: <br>
Logan Boras UCID: <br> 
Kenny Jeon UCID: 30068677

#### 1. Create directory for input data to live ####

Note that the deerfoot.csv file or whatever input data you are using must be within the data folder then proceed to store it on the hdfs

In [ ]:
!mkdir data
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data /inputs

#### 2. Grab the specific data we want from the data set ####

In [ ]:
file = '/tutInput/data/deerfoot.csv'

deerfootRDD = (sc.textFile(file, 8))
'\n'.join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0]))

#### 2a. Now we want to get the data only for morning and afternoon rush hour, 8am and 4pm will suffice ####

In [ ]:
def getRushHours(RDD):

    rushHours = RDD.split(',')
    return (rushHours[1],rushHours[6],rushHours[13])

In [ ]:
RushHoursRDD = deerfootRDD.map(getRushHours)